# Elevation Data Collection and Preprocessing

need to write something here


## About Modis (need to modify this later on)

MODIS (or Moderate Resolution Imaging Spectroradiometer) is a key instrument aboard the Terra (originally known as EOS AM-1) and Aqua (originally known as EOS PM-1) satellites. Terra's orbit around the Earth is timed so that it passes from north to south across the equator in the morning, while Aqua passes south to north over the equator in the afternoon. Terra MODIS and Aqua MODIS are viewing the entire Earth's surface every 1 to 2 days, acquiring data in 36 spectral bands, or groups of wavelengths (see MODIS Technical Specifications). These data will improve our understanding of global dynamics and processes occurring on the land, in the oceans, and in the lower atmosphere. MODIS is playing a vital role in the development of validated, global, interactive Earth system models able to predict global change accurately enough to assist policy makers in making sound decisions concerning the protection of our environment.

**Data Product Citations:**
Data Product Citations: 
NASA JPL (2013). NASA Shuttle Radar Topography Mission Global 3 arc second NetCDF. NASA EOSDIS Land Processes Distributed Active Archive Center. Accessed 2024-05-16 from https://doi.org/10.5067/MEaSUREs/SRTM/SRTMGL3_NC.003. Accessed May 16, 2024. 

## About AρρEEARS  (need to modify this later on) 
Application for Extracting and Exploring Analysis Ready Samples (AρρEEARS)

The Application for Extracting and Exploring Analysis Ready Samples (AρρEEARS) offers a simple and efficient way to access and transform geospatial data from a variety of federal data archives. AρρEEARS enables users to subset geospatial datasets using spatial, temporal, and band/layer parameters. Two types of sample requests are available: point samples for geographic coordinates and area samples for spatial areas via vector polygons. Sample requests submitted to AρρEEARS provide users not only with data values, but also associated quality data values. Interactive visualizations with summary statistics are provided for each sample within the application, which allow users to preview and interact with their samples before downloading their data. Get started with a sample request using the Extract option above, or visit the Help page to learn more. 


**Software Citation:**

Software Citation: 
AppEEARS Team. (2024). Application for Extracting and Exploring Analysis Ready Samples (AppEEARS). Ver. 3.54. NASA EOSDIS Land Processes Distributed Active Archive Center (LP DAAC), USGS/Earth Resources Observation and Science (EROS) Center, Sioux Falls, South Dakota, USA. Accessed May 16, 2024. https://appeears.earthdatacloud.nasa.gov 



[AppEEARS API](https://appeears.earthdatacloud.nasa.gov/api/)


In [1]:
#!pip install eccodes
#!pip install xarray
#!pip install rioxarray
#!pip install netCDF4
#!pip install ecmwflibs
#!pip install geopandas
#!pip install cftime

In [11]:
import rasterio
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rx
from scipy.io import netcdf
from netCDF4 import Dataset
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
import cftime
import matplotlib.pyplot as plt
import skimage.exposure
import seaborn as sns
import geogif
from IPython.display import Image
import datetime
from datetime import datetime
import requests
import json
import os
import time

## Helper Functions

In [12]:
def list_appeears_products():
    
    '''
    Retrieve and print a list of products available in the AppEEARS API.

    Returns:
        None

    Example:
        list_appeears_products()
    '''
    url = 'https://appeears.earthdatacloud.nasa.gov/api/product'
    response = requests.get(url)
    if response.status_code == 200:
        products = response.json()
        if products:
            for product in products:
                print(f"Product ID: {product['ProductAndVersion']}, Description: {product['Description']}")
        else:
            print("Failed to retrieve product list.")
    else:
        print(f"Failed to list products: {response.status_code}")


In [13]:
#list_appeears_products()

In [24]:
#import requests
product_id = 'SRTMGL3_NC.003'
response = requests.get('https://appeears.earthdatacloud.nasa.gov/api/product/{0}'.format(product_id))
layer_response = response.json()
print(layer_response)

{'SRTMGL3_DEM': {'AddOffset': '', 'Available': True, 'DataType': 'int16', 'Description': 'Elevation', 'Dimensions': ['time', 'lat', 'lon'], 'FillValue': -32768, 'FillValueAll': [-32768], 'Group': '', 'Info': {}, 'IsQA': False, 'Layer': 'SRTMGL3_DEM', 'OrigDataType': 'int16', 'OrigValidMax': 32767, 'OrigValidMin': -32767, 'QualityLayers': "['SRTMGL3_NUM']", 'QualityProductAndVersion': 'SRTMGL3_NUMNC.003', 'ScaleFactor': '', 'Units': 'Meters', 'ValidMax': 32767, 'ValidMin': -32767, 'XSize': 1201, 'YSize': 1201}}


In [15]:
def netcdf_to_dataframe(netcdf_file):
    
    '''
    Reads a netCDF file and converts its contents into a DataFrame.

    Parameters:
        netcdf_file (str): Path to the netCDF file.

    Returns:
        pandas.DataFrame: DataFrame containing the data from the netCDF file.
                          Each column represents a variable, and each row represents a time step.
    '''
    try:
        # Open the netCDF file using xarray
        ds = xr.open_dataset(netcdf_file,engine="netcdf4")

        # Convert the dataset to a pandas DataFrame
        df = ds.to_dataframe()

        return df

    except Exception as e:
        print("An error occurred:", str(e))
        return None


In [16]:
def convert_cftime_to_datetime64(cftime_dates):
    """ Convert an array of cftime dates to numpy datetime64."""
    return np.array([pd.Timestamp(str(date)) for date in cftime_dates])

In [17]:
def compute_stats(df, region_column_name):
    """
    Computes the maximum and variance of elevation data by region and district.

    Parameters:
        df (pandas.DataFrame): DataFrame containing elevation data.
        region_column_name (str): Column name for the region.

    Returns:
        pandas.DataFrame: DataFrame with computed statistics.
    """
    grouped = df.groupby([region_column_name, 'district'])['SRTMGL3_DEM']
    return grouped.agg(max_elevation='max', variance_elevation='var').reset_index()

In [18]:
def batch_data_extraction(nc_files, shapefile, output_directory):
    
    '''
    Process multiple NetCDF files, extract weather data, compute statistics for each district,
    and save the results to separate CSV files. Then merge all files into a single CSV file.

    Parameters:
        nc_files (list): List of paths to the NetCDF files.
        shapefile (str): Path to the shapefile.
        output_directory (str): Path to the directory where the output CSV files will be saved.
    '''
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Iterate over each NetCDF file
    for nc_file in nc_files:
        # Extract weather data from the NetCDF file
        ndvi_data = extract_ndvi_data(nc_file, shapefile)

        # Compute district ndvi mean statistics
        district_stats = process_ndvi_data(ndvi_data)

        # Generate output file name based on the year column
        year = district_stats['year'].iloc[1]  # Assuming all rows have the same year

        output_file = os.path.join(output_directory, f"ndvi_{year}.csv")

        # Save the district statistics to CSV
        district_stats.to_csv(output_file, index=False)
        print(f"The NDVI Data for year {year} has been successfully extracted and saved to:", output_file)

## Fetch Elevation Data From AppEEARS

In [19]:
def submit_task_and_fetch_data(geojson_file, start_date, end_date, product_id, layer_name, dest_dir):
    """
    Submit a task to fetch data using AppEEARS API and save the downloaded files in a directory based on the year.

    Args:
        geojson_file (str): Path to the GeoJSON file.
        start_date (str): Start date in MM-DD-YYYY format.
        end_date (str): End date in MM-DD-YYYY format.
        product_id (str): Product ID for the data to be fetched.
        layer_name (str): Name of the layer.
        dest_dir (str): Directory where files will be saved.

    Returns:
        None

    Example:
        submit_task_and_fetch_data('path/to/geojson_file.geojson', '01-01-2024', '12-31-2024', 'product_id', 'layer_name', 'destination_directory')
    """
    # Authenticate
    auth_url = 'https://appeears.earthdatacloud.nasa.gov/api/login'
    credentials = ('ashuu944', 'Hassan@16219')  # Use environment variables in production
    response = requests.post(auth_url, auth=credentials)
    if response.status_code != 200:
        print("Authentication failed.")
        return
    token = response.json()['token']

    # Read the GeoJSON file
    with open(geojson_file, 'r') as file:
        geojson = json.load(file)

    # Define the URL for submitting a task
    task_url = 'https://appeears.earthdatacloud.nasa.gov/api/task'
    headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'}
    task_params = {
        "task_type": "area",
        "task_name": "NDVI Data Extraction",
        "params": {
            "dates": [
                {
                    "startDate": start_date,
                    "endDate": end_date,
                    "recurring": False
                }
            ],
            "layers": [
                {
                    "product": product_id,
                    "layer": layer_name
                }
            ],
            "output": {
                "format": {
                    "type": "netcdf4"
                },
                "projection": "geographic"
            },
            "geo": geojson
        }
    }

    # Submit the task
    task_response = requests.post(task_url, headers=headers, json=task_params)
    task_id = task_response.json()['task_id']
    print(f"Task submitted, ID: {task_id}")

    # Check task status until it is done
    while True:
        time.sleep(50)  # Wait for 50 seconds before checking again
        status_response = requests.get(f'{task_url}/{task_id}', headers=headers)
        task_status = status_response.json()['status']
        print(f"Current task status: {task_status}")
        if task_status == 'done':
            print("Task completed. Proceeding to fetch file list.")
            break
        elif task_status in ['error', 'failed']:
            print("Task failed.")
            return

    # Fetch the bundle to get file IDs
    bundle_url = f'https://appeears.earthdatacloud.nasa.gov/api/bundle/{task_id}'
    bundle_response = requests.get(bundle_url, headers={'Authorization': f'Bearer {token}'})
    if bundle_response.status_code == 200:
        files = bundle_response.json()['files']
        # Get the year from start_date
        year = datetime.strptime(start_date, '%m-%d-%Y').year
        year_dir = os.path.join(dest_dir, str(year))
        os.makedirs(year_dir, exist_ok=True)
        for file in files:
            file_id = file['file_id']
            file_name = file['file_name']
            file_download_url = f'https://appeears.earthdatacloud.nasa.gov/api/bundle/{task_id}/{file_id}'
            download_response = requests.get(file_download_url, headers={'Authorization': f'Bearer {token}'}, stream=True)

            if download_response.status_code == 200:
                # Prefix the file name with the year
                year_prefixed_file_name = f"{year}_{file_name}"
                filepath = os.path.join(year_dir, year_prefixed_file_name)
                with open(filepath, 'wb') as f:
                    for chunk in download_response.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"Data downloaded successfully and saved to {filepath}")
            else:
                print(f"Failed to download file {file_name}: {download_response.status_code}, {download_response.text}")
    else:
        print(f"Failed to fetch file list: {bundle_response.status_code}, {bundle_response.text}")


## Extract the Elevation Data

In [20]:
def extract_elevation_data(nc_file, shapefile, save_dir):
    """
    Extracts elevation data from a NetCDF file for each geometry in a shapefile,
    computes the maximum and variance of the elevation data by district and region,
    and saves the results in a CSV file and a spatial data file.

    Parameters:
        nc_file (str): Path to the NetCDF file.
        shapefile (str): Path to the shapefile.
        save_dir (str): Directory where the output files will be saved.

    Returns:
        pandas.DataFrame: DataFrame containing extracted elevation data, excluding null values.
    """
    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Load the NetCDF file with xarray
    ds = xr.open_dataset(nc_file)
    ds.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)

    # Manually set the CRS if not defined
    if ds.rio.crs is None:
        ds.rio.write_crs("epsg:4326", inplace=True)

    # Load the shapefile with GeoPandas
    gdf = gpd.read_file(shapefile)

    # Convert GeoDataFrame CRS to match Dataset CRS
    if gdf.crs != ds.rio.crs:
        gdf = gdf.to_crs(ds.rio.crs)

    results = []

    # Determine the column name for administrative regions dynamically
    region_column_name = None
    for column_name in ['region', 'province']:
        if column_name in gdf.columns:
            region_column_name = column_name
            break

    if not region_column_name:
        raise ValueError("No administrative region column found in the shapefile.")

    # Loop through each geometry in the GeoDataFrame
    for idx, row in gdf.iterrows():
        geometry = row.geometry

        # Clip the dataset by the geometry
        clipped = ds.rio.clip([geometry], gdf.crs, drop=True, all_touched=True)

        # Process each clipped dataset
        if 'time' in clipped.dims:
            # Convert to DataFrame
            df = clipped.to_dataframe().reset_index()

            # Filter out rows where 'SRTMGL3_DEM' is null
            if 'SRTMGL3_DEM' in df.columns:
                df = df.dropna(subset=['SRTMGL3_DEM'])

            # Continue only if there are remaining rows after filtering
            if not df.empty:
                # Add region or province column
                df[region_column_name] = row[region_column_name]
                df['district'] = row['district']  # assuming 'district' column always exists
                results.append(df)
            else:
                print(f"No Elevation data for geometry at index {idx}. Skipping...")

        else:
            print(f"No data for geometry at index {idx} in the dataset.")

    # Combine all dataframes into a single DataFrame
    if results:
        result_df = pd.concat(results, ignore_index=True)
    else:
        result_df = pd.DataFrame()  # Return an empty DataFrame if no results

    # Compute statistics
    stats_df = compute_stats(result_df, region_column_name)

    # Save the results to a CSV file
    csv_path = os.path.join(save_dir, 'district_elevation_stats.csv')
    stats_df.to_csv(csv_path, index=False)

    # Merge statistics with the original GeoDataFrame
    merged_gdf = gdf.merge(stats_df, on=['district', region_column_name])

    # Save the spatial data with geometry
    geojson_path = os.path.join(save_dir, 'district_elevation_stats.geojson')
    merged_gdf.to_file(geojson_path, driver='GeoJSON')

    return result_df

## Download Tanzania Data

In [21]:
# Example usage
geojson_file = 'tanzania_data/shapefiles/tz_country.geojson'
start_date = '02-11-2000'
end_date   = '02-21-2000'
product_id = 'SRTMGL3_NC.003' 
layer_name = 'SRTMGL3_DEM'# 90m resolution layer

dest_dir = "tanzania_data/elevation_data/"
#submit_task_and_fetch_data(geojson_file, start_date, end_date, product_id, layer_name, dest_dir)

## Extra Data for each Districts

In [22]:
tz_dir = 'tanzania_data/elevation_data/'
nc_file = tz_dir+ '2000/2000_SRTMGL3_NC.003_90m_aid0001.nc'
shapefile = 'tanzania_data/shapefiles/tz_districts.shp'
output_directory = tz_dir + 'processed/'
#extr_data = extract_elevation_data(nc_file, shapefile, output_directory)

## Rwanda Data Download

In [8]:
# Example usage
geojson_file = 'rwanda_data/shapefiles/rw_country.geojson'
start_date = '02-11-2000'
end_date   = '02-21-2000'
product_id = 'SRTMGL3_NC.003' 
layer_name = 'SRTMGL3_DEM'# 90m resolution layer
dest_dir = "rwanda_data/elevation_data/"
#submit_task_and_fetch_data(geojson_file, start_date, end_date, product_id, layer_name, dest_dir)

## Extra Data for each Districts

In [17]:
rw_dir = 'rwanda_data/elevation_data/'
nc_file = rw_dir+ '2000/2000_SRTMGL3_NC.003_90m_aid0001.nc'
shapefile = 'rwanda_data/shapefiles/rw_district.shp'
output_directory = rw_dir + 'processed/'
extr_data = extract_elevation_data(nc_file, shapefile, output_directory)

In [18]:
extr_data.head()

,time,lat,lon,crs,SRTMGL3_DEM,province,district
0,2000-02-11 00:00:00,-1.308333,29.824167,0,2491.0,Amajyaruguru,Burera
1,2000-02-11 00:00:00,-1.309167,29.822500,0,2447.0,Amajyaruguru,Burera
2,2000-02-11 00:00:00,-1.309167,29.823333,0,2467.0,Amajyaruguru,Burera
3,2000-02-11 00:00:00,-1.309167,29.824167,0,2474.0,Amajyaruguru,Burera
4,2000-02-11 00:00:00,-1.309167,29.825000,0,2488.0,Amajyaruguru,Burera
